# RLDT

## Step 1: Import the necessary libraries:

In [136]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Step 2: Define the environment:

### Step 2.1: Devices

#### *Gloabl variables*

In [137]:
num_IOT_devices = 10

voltages_frequencies_IOT = [
    (1e6, 1.8),
    (2e6, 2.3),
    (4e6, 2.7),
    (8e6, 4.0),
    (16e6, 5.0),
    (32e6, 6.5),
]
num_MEC_devices = 5

voltages_frequencies_MEC = [
    (6 * 1e8, 0.8),
    (7.5 * 1e8, 0.825),
    (10 * 1e8, 1.0),
    (15 * 1e8, 1.2),
    (30 * 1e8, 2),
    (40 * 1e8, 3.1),
]

task_kinds = [1,2,3,4]

min_num_nodes_dag = 4
max_num_nodes_dag = 20
max_num_parents_dag = 5

num_dag_generations = 10000

#### *IOT*

In [138]:
devices_data_IOT = []
for i in range(num_IOT_devices):
    cpu_cores = np.random.choice([4, 6, 8])
    device_info = {
        "id": i,
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [np.random.choice([0, 1]) for _ in range(cpu_cores)],
        "voltages_frequencies": [
            voltages_frequencies_IOT[i]
            for i in np.random.choice(6, size=4, replace=False)
        ],
        "ISL": np.random.randint(10, 21),
        "capacitance": [np.random.uniform(2, 3) * 1e-9 for _ in range(cpu_cores)],
        "powerIdle": [
            np.random.choice([700, 800, 900]) * 1e-6 for _ in range(cpu_cores)
        ],
        "batteryLevel": np.random.randint(36, 41) * 1e9,
        "errorRate": np.random.randint(1, 6) / 100,
        "accetableTasks": np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        ),
        "handleSafeTask": np.random.choice([0, 1], p=[0.25, 0.75]),
    }
    devices_data_IOT.append(device_info)

IoTdevices = pd.DataFrame(devices_data_IOT)

IoTdevices.set_index("id", inplace=True)
IoTdevices

,number_of_cpu_cores,occupied_cores,voltages_frequencies,ISL,capacitance,powerIdle,batteryLevel,errorRate,accetableTasks,handleSafeTask
id,,,,,,,,,,
0,6,"[1, 0, 1, 1, 0, 1]","[(2000000.0, 2.3), (16000000.0, 5.0), (4000000...",18,"[2.8045171323141684e-09, 2.590786812582622e-09...","[0.0007999999999999999, 0.0007, 0.0007, 0.0007...",3.900000e+10,0.05,"[1, 3, 4, 2]",1
1,6,"[0, 0, 0, 0, 0, 1]","[(32000000.0, 6.5), (1000000.0, 1.8), (2000000...",18,"[2.000334788023672e-09, 2.684871812768801e-09,...","[0.0009, 0.0007, 0.0007, 0.0009, 0.0009, 0.0009]",3.700000e+10,0.04,"[3, 1]",1
2,4,"[0, 0, 0, 0]","[(4000000.0, 2.7), (8000000.0, 4.0), (16000000...",16,"[2.036000748245785e-09, 2.4697656169758026e-09...","[0.0007999999999999999, 0.0007, 0.0009, 0.0007]",3.900000e+10,0.05,"[3, 4, 1, 2]",0
3,4,"[0, 0, 0, 0]","[(32000000.0, 6.5), (2000000.0, 2.3), (1600000...",16,"[2.8121880202993396e-09, 2.7285173195324445e-0...","[0.0007, 0.0007, 0.0009, 0.0007999999999999999]",3.700000e+10,0.04,"[4, 3, 1]",1
4,8,"[1, 0, 0, 0, 0, 1, 0, 0]","[(2000000.0, 2.3), (32000000.0, 6.5), (1600000...",20,"[2.3303484372179294e-09, 2.156637689470351e-09...","[0.0007999999999999999, 0.0009, 0.000799999999...",3.900000e+10,0.03,"[3, 4]",1
5,4,"[1, 1, 0, 1]","[(8000000.0, 4.0), (32000000.0, 6.5), (4000000...",18,"[2.469862490866213e-09, 2.471311553690515e-09,...","[0.0007999999999999999, 0.0007999999999999999,...",4.000000e+10,0.02,"[2, 3, 1]",1
6,8,"[0, 1, 0, 1, 0, 1, 0, 1]","[(32000000.0, 6.5), (2000000.0, 2.3), (1600000...",12,"[2.2195158166067312e-09, 2.1846467158445587e-0...","[0.0009, 0.0009, 0.0007, 0.0007, 0.00079999999...",3.900000e+10,0.03,"[4, 3, 2]",0
7,8,"[1, 1, 1, 0, 0, 1, 1, 1]","[(4000000.0, 2.7), (1000000.0, 1.8), (8000000....",18,"[2.2533324161914662e-09, 2.440030718634443e-09...","[0.0009, 0.0007999999999999999, 0.000799999999...",3.600000e+10,0.03,"[4, 2]",1
8,4,"[1, 0, 1, 0]","[(8000000.0, 4.0), (16000000.0, 5.0), (2000000...",19,"[2.7179231959272954e-09, 2.8725222466545013e-0...","[0.0007999999999999999, 0.0007999999999999999,...",3.700000e+10,0.02,"[1, 3]",0


#### *MEC*

In [139]:
devices_data_MEC = []
for i in range(num_MEC_devices):
    cpu_cores = np.random.choice([16,32,64])
    device_info = {
        "id": i,
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [np.random.choice([0, 1]) for _ in range(cpu_cores)],
        "voltages_frequencies": [
            voltages_frequencies_MEC[i]
            for i in np.random.choice(6, size=4, replace=False)
        ],
        "capacitance": [np.random.uniform(1.5, 2) * 1e-9 for _ in range(cpu_cores)],
        "powerIdle": [np.random.choice([9, 9, 10]) * 1e-5 for _ in range(cpu_cores)],
        "errorRate": np.random.randint(5, 11) / 100,
        "accetableTasks": np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        ),
        "handleSafeTask": np.random.choice([0, 1], p=[0.75, 0.25]),
        "batteryLevel": 100,
        "ISL": 0,
    }
    devices_data_MEC.append(device_info)

MECDevices = pd.DataFrame(devices_data_MEC)

MECDevices.set_index("id", inplace=True)
MECDevices

,number_of_cpu_cores,occupied_cores,voltages_frequencies,capacitance,powerIdle,errorRate,accetableTasks,handleSafeTask,batteryLevel,ISL
id,,,,,,,,,,
0,32,"[1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, ...","[(1000000000.0, 1.0), (750000000.0, 0.825), (1...","[1.6464760715534162e-09, 1.7712662574401735e-0...","[9e-05, 0.0001, 9e-05, 0.0001, 9e-05, 0.0001, ...",0.07,"[3, 2]",0,100,0
1,32,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[(3000000000.0, 2), (1000000000.0, 1.0), (4000...","[1.8658941981728273e-09, 1.848729443166088e-09...","[0.0001, 9e-05, 0.0001, 9e-05, 0.0001, 0.0001,...",0.09,"[2, 4, 3, 1]",0,100,0
2,64,"[0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, ...","[(3000000000.0, 2), (4000000000.0, 3.1), (1500...","[1.541924453625214e-09, 1.706608988567053e-09,...","[9e-05, 9e-05, 9e-05, 9e-05, 0.0001, 0.0001, 0...",0.06,"[3, 1]",0,100,0
3,64,"[1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, ...","[(1000000000.0, 1.0), (750000000.0, 0.825), (1...","[1.8673535140196998e-09, 1.9239181372871347e-0...","[0.0001, 0.0001, 9e-05, 9e-05, 9e-05, 0.0001, ...",0.08,"[4, 2, 1]",1,100,0
4,32,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, ...","[(4000000000.0, 3.1), (600000000.0, 0.8), (750...","[1.662249822420567e-09, 1.993030626781426e-09,...","[9e-05, 0.0001, 9e-05, 9e-05, 9e-05, 9e-05, 9e...",0.10,"[2, 4, 3]",0,100,0


#### *CLOUD*

In [142]:
cloud_configurations = [(1,13.85), (2,3.9e9,24.28)]
device_info = [
    {
        "id": 0,
        "number_of_cpu_cores": 1,
        "occupied_cores": [0],
        "voltages_frequencies": [2.8e9, 3.9e9],
        "capacitance": (13.85, 24.28),
        "powerIdle": 0,
        "ISL": 0,
        "batteryLevel": 100,
        "errorRate": 0.1,
        "accetableTasks": [1, 2, 3, 4],
        "handleSafeTask": 0,
    }
]
cloud = pd.DataFrame(device_info)
cloud=cloud.set_index("id")
cloud

,number_of_cpu_cores,occupied_cores,voltages_frequencies,capacitance,powerIdle,ISL,batteryLevel,errorRate,accetableTasks,handleSafeTask
id,,,,,,,,,,
0,1,[0],"[2800000000.0, 3900000000.0]","(13.85, 24.28)",0,0,100,0.1,"[1, 2, 3, 4]",0


####        ALL THE DEVICES

In [160]:
IoTdevices

,number_of_cpu_cores,occupied_cores,voltages_frequencies,ISL,capacitance,powerIdle,batteryLevel,errorRate,accetableTasks,handleSafeTask
id,,,,,,,,,,
0,6,"[1, 0, 1, 1, 0, 1]","[(2000000.0, 2.3), (16000000.0, 5.0), (4000000...",18,"[2.8045171323141684e-09, 2.590786812582622e-09...","[0.0007999999999999999, 0.0007, 0.0007, 0.0007...",3.900000e+10,0.05,"[1, 3, 4, 2]",1
1,6,"[0, 0, 0, 0, 0, 1]","[(32000000.0, 6.5), (1000000.0, 1.8), (2000000...",18,"[2.000334788023672e-09, 2.684871812768801e-09,...","[0.0009, 0.0007, 0.0007, 0.0009, 0.0009, 0.0009]",3.700000e+10,0.04,"[3, 1]",1
2,4,"[0, 0, 0, 0]","[(4000000.0, 2.7), (8000000.0, 4.0), (16000000...",16,"[2.036000748245785e-09, 2.4697656169758026e-09...","[0.0007999999999999999, 0.0007, 0.0009, 0.0007]",3.900000e+10,0.05,"[3, 4, 1, 2]",0
3,4,"[0, 0, 0, 0]","[(32000000.0, 6.5), (2000000.0, 2.3), (1600000...",16,"[2.8121880202993396e-09, 2.7285173195324445e-0...","[0.0007, 0.0007, 0.0009, 0.0007999999999999999]",3.700000e+10,0.04,"[4, 3, 1]",1
4,8,"[1, 0, 0, 0, 0, 1, 0, 0]","[(2000000.0, 2.3), (32000000.0, 6.5), (1600000...",20,"[2.3303484372179294e-09, 2.156637689470351e-09...","[0.0007999999999999999, 0.0009, 0.000799999999...",3.900000e+10,0.03,"[3, 4]",1
5,4,"[1, 1, 0, 1]","[(8000000.0, 4.0), (32000000.0, 6.5), (4000000...",18,"[2.469862490866213e-09, 2.471311553690515e-09,...","[0.0007999999999999999, 0.0007999999999999999,...",4.000000e+10,0.02,"[2, 3, 1]",1
6,8,"[0, 1, 0, 1, 0, 1, 0, 1]","[(32000000.0, 6.5), (2000000.0, 2.3), (1600000...",12,"[2.2195158166067312e-09, 2.1846467158445587e-0...","[0.0009, 0.0009, 0.0007, 0.0007, 0.00079999999...",3.900000e+10,0.03,"[4, 3, 2]",0
7,8,"[1, 1, 1, 0, 0, 1, 1, 1]","[(4000000.0, 2.7), (1000000.0, 1.8), (8000000....",18,"[2.2533324161914662e-09, 2.440030718634443e-09...","[0.0009, 0.0007999999999999999, 0.000799999999...",3.600000e+10,0.03,"[4, 2]",1
8,4,"[1, 0, 1, 0]","[(8000000.0, 4.0), (16000000.0, 5.0), (2000000...",19,"[2.7179231959272954e-09, 2.8725222466545013e-0...","[0.0007999999999999999, 0.0007999999999999999,...",3.700000e+10,0.02,"[1, 3]",0


In [144]:
devices = pd.concat([IoTdevices,MECDevices,cloud],ignore_index=True)
devices

,number_of_cpu_cores,occupied_cores,voltages_frequencies,ISL,capacitance,powerIdle,batteryLevel,errorRate,accetableTasks,handleSafeTask
0,6,"[1, 0, 1, 1, 0, 1]","[(2000000.0, 2.3), (16000000.0, 5.0), (4000000...",18,"[2.8045171323141684e-09, 2.590786812582622e-09...","[0.0007999999999999999, 0.0007, 0.0007, 0.0007...",3.900000e+10,0.05,"[1, 3, 4, 2]",1
1,6,"[0, 0, 0, 0, 0, 1]","[(32000000.0, 6.5), (1000000.0, 1.8), (2000000...",18,"[2.000334788023672e-09, 2.684871812768801e-09,...","[0.0009, 0.0007, 0.0007, 0.0009, 0.0009, 0.0009]",3.700000e+10,0.04,"[3, 1]",1
2,4,"[0, 0, 0, 0]","[(4000000.0, 2.7), (8000000.0, 4.0), (16000000...",16,"[2.036000748245785e-09, 2.4697656169758026e-09...","[0.0007999999999999999, 0.0007, 0.0009, 0.0007]",3.900000e+10,0.05,"[3, 4, 1, 2]",0
3,4,"[0, 0, 0, 0]","[(32000000.0, 6.5), (2000000.0, 2.3), (1600000...",16,"[2.8121880202993396e-09, 2.7285173195324445e-0...","[0.0007, 0.0007, 0.0009, 0.0007999999999999999]",3.700000e+10,0.04,"[4, 3, 1]",1
4,8,"[1, 0, 0, 0, 0, 1, 0, 0]","[(2000000.0, 2.3), (32000000.0, 6.5), (1600000...",20,"[2.3303484372179294e-09, 2.156637689470351e-09...","[0.0007999999999999999, 0.0009, 0.000799999999...",3.900000e+10,0.03,"[3, 4]",1
5,4,"[1, 1, 0, 1]","[(8000000.0, 4.0), (32000000.0, 6.5), (4000000...",18,"[2.469862490866213e-09, 2.471311553690515e-09,...","[0.0007999999999999999, 0.0007999999999999999,...",4.000000e+10,0.02,"[2, 3, 1]",1
6,8,"[0, 1, 0, 1, 0, 1, 0, 1]","[(32000000.0, 6.5), (2000000.0, 2.3), (1600000...",12,"[2.2195158166067312e-09, 2.1846467158445587e-0...","[0.0009, 0.0009, 0.0007, 0.0007, 0.00079999999...",3.900000e+10,0.03,"[4, 3, 2]",0
7,8,"[1, 1, 1, 0, 0, 1, 1, 1]","[(4000000.0, 2.7), (1000000.0, 1.8), (8000000....",18,"[2.2533324161914662e-09, 2.440030718634443e-09...","[0.0009, 0.0007999999999999999, 0.000799999999...",3.600000e+10,0.03,"[4, 2]",1
8,4,"[1, 0, 1, 0]","[(8000000.0, 4.0), (16000000.0, 5.0), (2000000...",19,"[2.7179231959272954e-09, 2.8725222466545013e-0...","[0.0007999999999999999, 0.0007999999999999999,...",3.700000e+10,0.02,"[1, 3]",0
9,8,"[1, 0, 0, 1, 1, 0, 0, 1]","[(32000000.0, 6.5), (8000000.0, 4.0), (2000000...",17,"[2.892297963493987e-09, 2.114703605898902e-09,...","[0.0007999999999999999, 0.0007, 0.0007, 0.0007...",3.800000e+10,0.05,"[4, 3, 2]",1


### Step 2.2: Application

#### *helper function : generate_random_dag*

In [145]:
def generate_random_dag(num_nodes):
    dag = nx.DiGraph()

    nodes = [f"t{i+1}" for i in range(num_nodes)]
    dag.add_nodes_from(nodes)

    available_parents = {node: list(nodes[:i]) for i, node in enumerate(nodes)}

    for i in range(2, num_nodes + 1):
       
        num_parents = min(
            random.randint(1, min(i, max_num_parents_dag)), len(available_parents[f"t{i}"])
        )

        # select parents
        parent_nodes = random.sample(available_parents[f"t{i}"], num_parents)
        # add parents
        dag.add_edges_from((parent_node, f"t{i}") for parent_node in parent_nodes)

        # update available parents
        available_parents[f"t{i}"] = list(nodes[:i])

    return dag

#### *Generate task DAGs*

In [146]:
tasks_data = []

start_node_number = 1
for run in range(num_dag_generations):

    num_nodes = random.randint(min_num_nodes_dag, max_num_nodes_dag)

    random_dag = generate_random_dag(num_nodes)

    mapping = {
        f"t{i}": f"t{i + start_node_number - 1}" for i in range(1, num_nodes + 1)
    }

    random_dag = nx.relabel_nodes(random_dag, mapping)

    for node in random_dag.nodes:
        parents = list(random_dag.predecessors(node))
        task_info = {
            "id": node,
            "dependency": parents,
            "mobility": np.random.randint(1, 10),
            "kind": np.random.choice(task_kinds),
            "safe": np.random.choice([0, 1], p=[0.95, 0.05]),
            "computationalLoad": int(np.random.uniform(1, 100) * 1e4),
            "dataEntrySize": (np.random.randint(10, 100) // 10)
            * (10 ** np.random.choice([3, 6])),
            "returnDataSize": (np.random.randint(10, 100) // 10)
            * (10 ** np.random.choice([3, 6])),
            "status": "READY",
        }
        tasks_data.append(task_info)
    start_node_number += num_nodes

np.random.shuffle(tasks_data)
tasks = pd.DataFrame(tasks_data)

tasks.set_index("id", inplace=True)

tasks

,dependency,mobility,kind,safe,computationalLoad,dataEntrySize,returnDataSize,status
id,,,,,,,,
t27584,[t27575],3,4,0,199555,1000,5000,READY
t49074,"[t49070, t49071, t49073]",7,2,0,366134,3000000,9000,READY
t57600,"[t57593, t57595]",7,2,0,540034,4000000,9000000,READY
t80041,[t80040],6,4,0,748536,3000,3000,READY
t15317,[t15316],9,4,0,961058,8000,4000,READY
...,...,...,...,...,...,...,...,...
t37601,"[t37598, t37600]",3,4,0,932851,5000000,6000000,READY
t71294,[t71293],2,2,0,504963,1000000,1000,READY
t9638,[t9635],4,2,0,134904,2000,7000000,READY


In [147]:
tasks

,dependency,mobility,kind,safe,computationalLoad,dataEntrySize,returnDataSize,status
id,,,,,,,,
t27584,[t27575],3,4,0,199555,1000,5000,READY
t49074,"[t49070, t49071, t49073]",7,2,0,366134,3000000,9000,READY
t57600,"[t57593, t57595]",7,2,0,540034,4000000,9000000,READY
t80041,[t80040],6,4,0,748536,3000,3000,READY
t15317,[t15316],9,4,0,961058,8000,4000,READY
...,...,...,...,...,...,...,...,...
t37601,"[t37598, t37600]",3,4,0,932851,5000000,6000000,READY
t71294,[t71293],2,2,0,504963,1000000,1000,READY
t9638,[t9635],4,2,0,134904,2000,7000000,READY


## Step 3: Preprocessing

### Step 3.1: Clustering

In [150]:
devices

,number_of_cpu_cores,occupied_cores,voltages_frequencies,ISL,capacitance,powerIdle,batteryLevel,errorRate,accetableTasks,handleSafeTask
0,6,"[1, 0, 1, 1, 0, 1]","[(2000000.0, 2.3), (16000000.0, 5.0), (4000000...",18,"[2.8045171323141684e-09, 2.590786812582622e-09...","[0.0007999999999999999, 0.0007, 0.0007, 0.0007...",3.900000e+10,0.05,"[1, 3, 4, 2]",1
1,6,"[0, 0, 0, 0, 0, 1]","[(32000000.0, 6.5), (1000000.0, 1.8), (2000000...",18,"[2.000334788023672e-09, 2.684871812768801e-09,...","[0.0009, 0.0007, 0.0007, 0.0009, 0.0009, 0.0009]",3.700000e+10,0.04,"[3, 1]",1
2,4,"[0, 0, 0, 0]","[(4000000.0, 2.7), (8000000.0, 4.0), (16000000...",16,"[2.036000748245785e-09, 2.4697656169758026e-09...","[0.0007999999999999999, 0.0007, 0.0009, 0.0007]",3.900000e+10,0.05,"[3, 4, 1, 2]",0
3,4,"[0, 0, 0, 0]","[(32000000.0, 6.5), (2000000.0, 2.3), (1600000...",16,"[2.8121880202993396e-09, 2.7285173195324445e-0...","[0.0007, 0.0007, 0.0009, 0.0007999999999999999]",3.700000e+10,0.04,"[4, 3, 1]",1
4,8,"[1, 0, 0, 0, 0, 1, 0, 0]","[(2000000.0, 2.3), (32000000.0, 6.5), (1600000...",20,"[2.3303484372179294e-09, 2.156637689470351e-09...","[0.0007999999999999999, 0.0009, 0.000799999999...",3.900000e+10,0.03,"[3, 4]",1
5,4,"[1, 1, 0, 1]","[(8000000.0, 4.0), (32000000.0, 6.5), (4000000...",18,"[2.469862490866213e-09, 2.471311553690515e-09,...","[0.0007999999999999999, 0.0007999999999999999,...",4.000000e+10,0.02,"[2, 3, 1]",1
6,8,"[0, 1, 0, 1, 0, 1, 0, 1]","[(32000000.0, 6.5), (2000000.0, 2.3), (1600000...",12,"[2.2195158166067312e-09, 2.1846467158445587e-0...","[0.0009, 0.0009, 0.0007, 0.0007, 0.00079999999...",3.900000e+10,0.03,"[4, 3, 2]",0
7,8,"[1, 1, 1, 0, 0, 1, 1, 1]","[(4000000.0, 2.7), (1000000.0, 1.8), (8000000....",18,"[2.2533324161914662e-09, 2.440030718634443e-09...","[0.0009, 0.0007999999999999999, 0.000799999999...",3.600000e+10,0.03,"[4, 2]",1
8,4,"[1, 0, 1, 0]","[(8000000.0, 4.0), (16000000.0, 5.0), (2000000...",19,"[2.7179231959272954e-09, 2.8725222466545013e-0...","[0.0007999999999999999, 0.0007999999999999999,...",3.700000e+10,0.02,"[1, 3]",0
9,8,"[1, 0, 0, 1, 1, 0, 0, 1]","[(32000000.0, 6.5), (8000000.0, 4.0), (2000000...",17,"[2.892297963493987e-09, 2.114703605898902e-09,...","[0.0007999999999999999, 0.0007, 0.0007, 0.0007...",3.800000e+10,0.05,"[4, 3, 2]",1


In [157]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


def calculate_mean_voltage_frequency(x):
    if len(x) == 2:
        return (x[0] + x[1]) / 2
    return sum([vf[0] / vf[1] for vf in x]) / len(x)


selected_features = [
    "number_of_cpu_cores",
    "voltages_frequencies",
    "capacitance",
    "ISL",
    "batteryLevel",
    "errorRate",
    "accetableTasks",
]

X = devices[selected_features].copy()
X["mean_voltage_frequency"] = X["voltages_frequencies"].apply(
    calculate_mean_voltage_frequency
)
X["mean_capacitance"] = X["capacitance"].apply(lambda x: sum(x) / len(x))
X["meanBattery"] = X["batteryLevel"] * (1 - X["ISL"])
X["doesAcceptT1"] = X["accetableTasks"].apply(lambda x: 1 in x)
X["doesAcceptT2"] = X["accetableTasks"].apply(lambda x: 2 in x)
X["doesAcceptT3"] = X["accetableTasks"].apply(lambda x: 3 in x)
X["doesAcceptT4"] = X["accetableTasks"].apply(lambda x: 4 in x)

X.drop(
    ["voltages_frequencies", "capacitance", "batteryLevel", "ISL", "accetableTasks"],
    axis=1,
    inplace=True,
)


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

n_clusters = 8

kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=42)
kmeans.fit(X_scaled)
cluster_labels = kmeans.labels_

devices["cluster"] = cluster_labels
devices

,number_of_cpu_cores,occupied_cores,voltages_frequencies,ISL,capacitance,powerIdle,batteryLevel,errorRate,accetableTasks,handleSafeTask,cluster
0,6,"[1, 0, 1, 1, 0, 1]","[(2000000.0, 2.3), (16000000.0, 5.0), (4000000...",18,"[2.8045171323141684e-09, 2.590786812582622e-09...","[0.0007999999999999999, 0.0007, 0.0007, 0.0007...",3.900000e+10,0.05,"[1, 3, 4, 2]",1,3
1,6,"[0, 0, 0, 0, 0, 1]","[(32000000.0, 6.5), (1000000.0, 1.8), (2000000...",18,"[2.000334788023672e-09, 2.684871812768801e-09,...","[0.0009, 0.0007, 0.0007, 0.0009, 0.0009, 0.0009]",3.700000e+10,0.04,"[3, 1]",1,0
2,4,"[0, 0, 0, 0]","[(4000000.0, 2.7), (8000000.0, 4.0), (16000000...",16,"[2.036000748245785e-09, 2.4697656169758026e-09...","[0.0007999999999999999, 0.0007, 0.0009, 0.0007]",3.900000e+10,0.05,"[3, 4, 1, 2]",0,3
3,4,"[0, 0, 0, 0]","[(32000000.0, 6.5), (2000000.0, 2.3), (1600000...",16,"[2.8121880202993396e-09, 2.7285173195324445e-0...","[0.0007, 0.0007, 0.0009, 0.0007999999999999999]",3.700000e+10,0.04,"[4, 3, 1]",1,3
4,8,"[1, 0, 0, 0, 0, 1, 0, 0]","[(2000000.0, 2.3), (32000000.0, 6.5), (1600000...",20,"[2.3303484372179294e-09, 2.156637689470351e-09...","[0.0007999999999999999, 0.0009, 0.000799999999...",3.900000e+10,0.03,"[3, 4]",1,4
5,4,"[1, 1, 0, 1]","[(8000000.0, 4.0), (32000000.0, 6.5), (4000000...",18,"[2.469862490866213e-09, 2.471311553690515e-09,...","[0.0007999999999999999, 0.0007999999999999999,...",4.000000e+10,0.02,"[2, 3, 1]",1,0
6,8,"[0, 1, 0, 1, 0, 1, 0, 1]","[(32000000.0, 6.5), (2000000.0, 2.3), (1600000...",12,"[2.2195158166067312e-09, 2.1846467158445587e-0...","[0.0009, 0.0009, 0.0007, 0.0007, 0.00079999999...",3.900000e+10,0.03,"[4, 3, 2]",0,4
7,8,"[1, 1, 1, 0, 0, 1, 1, 1]","[(4000000.0, 2.7), (1000000.0, 1.8), (8000000....",18,"[2.2533324161914662e-09, 2.440030718634443e-09...","[0.0009, 0.0007999999999999999, 0.000799999999...",3.600000e+10,0.03,"[4, 2]",1,6
8,4,"[1, 0, 1, 0]","[(8000000.0, 4.0), (16000000.0, 5.0), (2000000...",19,"[2.7179231959272954e-09, 2.8725222466545013e-0...","[0.0007999999999999999, 0.0007999999999999999,...",3.700000e+10,0.02,"[1, 3]",0,0
9,8,"[1, 0, 0, 1, 1, 0, 0, 1]","[(32000000.0, 6.5), (8000000.0, 4.0), (2000000...",17,"[2.892297963493987e-09, 2.114703605898902e-09,...","[0.0007999999999999999, 0.0007, 0.0007, 0.0007...",3.800000e+10,0.05,"[4, 3, 2]",1,4


## Step 4 : DDT

### Step 4.1:  Initializing The tree

In [195]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class YourModel(nn.Module):
    def __init__(self, num_clusters):
        super(YourModel, self).__init__()
        self.num_clusters = num_clusters
        self.prob_distribution = nn.Parameter(torch.zeros(self.num_clusters))

    def forward(self, input):
        # Access the parameter values
        prob_values = self.prob_distribution

        # Convert values to a probability distribution using softmax
        prob_distribution = F.softmax(prob_values, dim=0)

        # Sample an element from the probability distribution
        selected_element_index = torch.multinomial(prob_distribution, 1)

        # Get the selected element value
        selected_element_value = prob_values[selected_element_index]

        return selected_element_index, prob_distribution


# Example usage
num_clusters = 5
model = YourModel(num_clusters)

# Assuming you have some input
input = torch.randn(3, 4)  # Example input, adjust the shape as needed

# Forward pass
selected_element_value, prob_distribution = model(input)

print("Selected element value:", selected_element_value.item())
print("Probability distribution:", prob_distribution)

Selected element value: 4
Probability distribution: tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000], grad_fn=<SoftmaxBackward0>)


In [197]:
devices[devices["cluster"] == 4]

,number_of_cpu_cores,occupied_cores,voltages_frequencies,ISL,capacitance,powerIdle,batteryLevel,errorRate,accetableTasks,handleSafeTask,cluster
4,8,"[1, 0, 0, 0, 0, 1, 0, 0]","[(2000000.0, 2.3), (32000000.0, 6.5), (1600000...",20,"[2.3303484372179294e-09, 2.156637689470351e-09...","[0.0007999999999999999, 0.0009, 0.000799999999...",3.900000e+10,0.03,"[3, 4]",1,4
6,8,"[0, 1, 0, 1, 0, 1, 0, 1]","[(32000000.0, 6.5), (2000000.0, 2.3), (1600000...",12,"[2.2195158166067312e-09, 2.1846467158445587e-0...","[0.0009, 0.0009, 0.0007, 0.0007, 0.00079999999...",3.900000e+10,0.03,"[4, 3, 2]",0,4
9,8,"[1, 0, 0, 1, 1, 0, 0, 1]","[(32000000.0, 6.5), (8000000.0, 4.0), (2000000...",17,"[2.892297963493987e-09, 2.114703605898902e-09,...","[0.0007999999999999999, 0.0007, 0.0007, 0.0007...",3.800000e+10,0.05,"[4, 3, 2]",1,4


In [158]:
class DDT(nn.Module):
    def __init__(self, feature_size, num_clusters, depth, max_depth):
        super(DDT, self).__init__()
        self.feature_size = feature_size
        self.num_clusters = num_clusters
        self.depth = depth
        self.max_depth = max_depth
        self.selected_cluster = None
        self.selected_device = None
        self.selected_core = None
        self.selected_dvfs = None
        self.num_devices =0
        self.num_cores =0
        self.num_vf =0

        self.weight = nn.Parameter(torch.randn(feature_size))
        self.bias = nn.Parameter(torch.randn(1))
        self.alpha =  nn.Parameter(torch.randn(1))
        if depth ==3:
            self.prob_distribution = nn.Parameter(torch.zeros(self.num_clusters))
            self.selected_cluster = torch.multinomial(F.softmax(self.prob_distribution, dim=0), 1)
            self.num_devices = len(devices[devices["cluster"] == self.selected_cluster])
        elif depth ==5:
            self.prob_distribution = nn.Parameter(torch.zeros(self.num_devices))
            self.selected_device = torch.multinomial(F.softmax(self.prob_distribution, dim=0), 1)
            device = devices[devices["cluster"] == self.selected_cluster].iloc[self.selected_device]
            self.num_cores = len(device['number_of_cpu_cores'])
        elif depth ==7:
            self.prob_distribution = nn.Parameter(torch.zeros(self.num_cores))
            self.selected_core = torch.multinomial(F.softmax(self.prob_distribution, dim=0), 1)
            device = devices[devices["cluster"] == self.selected_cluster].iloc[self.selected_device]
            self.num_vf = len(device["voltages_frequencies"][self.selected_core])
        elif depth ==9:
            self.prob_distribution = nn.Parameter(torch.zeros(self.vf))
            self.selected_vf = torch.multinomial(F.softmax(self.prob_distribution, dim=0), 1)

        if self.depth <= self.max_depth:
            self.left_child = DDT(feature_size, num_clusters, depth+1, max_depth)
            self.right_child = DDT(feature_size, num_clusters, depth + 1, max_depth)

    def forward(self, x):
        if self.depth == self.max_depth:
            return self.prob_distribution
        else:
            decisions = torch.sigmoid(torch.matmul(x, self.weight) + self.bias)
            batch_results = torch.zeros(x.size(0), self.num_classes)
            for i in range(x.size(0)):
                decision = decisions[i]
                if decision > 0.5:
                    batch_results[i] = self.right_child(x[i:i+1])
                else:
                    batch_results[i] = self.left_child(x[i:i+1])
            return batch_results

In [ ]:
import torch.nn.functional as F


class DDT(nn.Module):
    def __init__(self, feature_size, num_clusters, depth, max_depth):
        super(DDT, self).__init__()
        self.feature_size = feature_size
        self.num_clusters = num_clusters
        self.depth = depth
        self.max_depth = max_depth
        self.selected_cluster = None
        self.selected_device = None
        self.selected_core = None
        self.selected_vf = None
        self.num_devices = 0
        self.num_cores = 0
        self.num_vf = 0

        self.weight = nn.Parameter(torch.randn(feature_size))
        self.bias = nn.Parameter(torch.randn(1))
        self.alpha = nn.Parameter(torch.randn(1))

        if self.depth < self.max_depth:
            self.left_child = DDT(feature_size, num_clusters, depth + 1, max_depth)
            self.right_child = DDT(feature_size, num_clusters, depth + 1, max_depth)

    def forward(self, x, devices):
        decision_score = torch.dot(x, self.weight) + self.bias
        decision_score = self.alpha * decision_score


        if self.depth == 3:
            prob_distribution = F.softmax(self.prob_distribution, dim=0)
            self.selected_cluster = torch.multinomial(prob_distribution, 1)
            self.num_devices = len(
                devices[devices["cluster"] == self.selected_cluster.item()]
            )
            return self.left_child(x, devices), self.right_child(x, devices)

        elif self.depth == 5:
            prob_distribution = F.softmax(self.prob_distribution, dim=0)
            self.selected_device = torch.multinomial(prob_distribution, 1)
            device = devices[devices["cluster"] == self.selected_cluster.item()].iloc[
                self.selected_device.item()
            ]
            self.num_cores = len(device["number_of_cpu_cores"])
            return self.left_child(x, devices), self.right_child(x, devices)

        elif self.depth == 7:
            prob_distribution = F.softmax(self.prob_distribution, dim=0)
            self.selected_core = torch.multinomial(prob_distribution, 1)
            device = devices[devices["cluster"] == self.selected_cluster.item()].iloc[
                self.selected_device.item()
            ]
            self.num_vf = len(device["voltages_frequencies"][self.selected_core.item()])
            return self.left_child(x, devices), self.right_child(x, devices)

        elif self.depth == 9:
            prob_distribution = F.softmax(self.prob_distribution, dim=0)
            self.selected_vf = torch.multinomial(prob_distribution, 1)
            return self.left_child(x, devices), self.right_child(x, devices)

## Step 5: RL

In [96]:
def getStatusOfDependcy(dependency):
    result = 0
    for dep in dependency:
        result += tasks_copy.loc[dep, "status"]
    return result


tasks_copy = tasks.copy()
tasks_copy["status"] = tasks_copy["status"].map(
    {"WAIT": 0, "READY": 1, "QUEUED": 2, "SCEDULED": 3}
)
tasks_copy["dependency"] = tasks_copy["dependency"].apply(getStatusOfDependcy)

taskList = tasks_copy.index.tolist()
devices_copy = devices.copy()

In [127]:
def calc_execution_time(device, task, dvfs):
    return task["computationalLoad"] / device["voltages_frequencies"][dvfs][0]


def calc_power_consumption(device,task,dvfs):
    return device["capacitance"][dvfs] * (device["voltages_frequencies"][dvfs][1] **2 )* calc_execution_time(device, task, dvfs)


def getTraverseProbabilty(tree, x):
    if tree.depth == tree.max_depth:
        return 1
    epsilon = 1e-7 
    decision = torch.sigmoid(torch.matmul(x, tree.weight) + tree.bias) + epsilon
    if decision > 0.5:
        return decision * getTraverseProbabilty(tree.right_child, x)
    else:
        return decision * getTraverseProbabilty(tree.left_child, x)

In [159]:
tasks_copy

,dependency,mobility,kind,safe,computationalLoad,dataEntrySize,returnDataSize,status
id,,,,,,,,
t43616,1,4,2,0,612071,3000000,5000000,1
t112909,4,1,2,0,384525,2000000,4000000,1
t2223,3,1,1,0,831229,2000,4000000,1
t36282,3,8,3,0,542845,7000000,5000000,1
t34491,1,3,3,0,519047,8000,4000,1
...,...,...,...,...,...,...,...,...
t40987,4,8,1,0,381646,4000,9000000,1
t66875,3,3,1,0,457769,9000000,7000000,1
t88425,1,6,2,0,21308,2000000,7000000,1


In [154]:
taskList = tasks_copy.index.tolist()

class Environment:
    def __init__(self):
        self.feature_size = 8
        self.num_actions = 8
        self.max_depth = 3 
        self.agent = DDTNode(self.feature_size, self.num_actions, depth=0, max_depth=self.max_depth)
        self.optimizer = optim.Adam(self.agent.parameters(), lr=0.005)

    def execute_action(self,state,action):
        taskList.pop(0)

        selected_cluster = devices_copy[devices_copy["cluster"] == 5]
        for row, device in selected_cluster.iterrows():
            checkAvailableCoree = (sum(device["occupied_cores"]) != device["number_of_cpu_cores"])
            checkIfSutible = state['kind'] in device['accetableTasks'] and state['safe']== device['handleSafeTask']
            checkBattery = device['batteryLevel'] > device['ISL']*2e8
            
            if (
                checkAvailableCoree
                and checkIfSutible
                and checkBattery
                and np.random.rand() > device['errorRate']
            ):
                
                e = calc_power_consumption(device, state, 0)
                t = calc_execution_time(device, state, 0)

                return (tasks_copy.loc[taskList[0]], -1 * t + -1 * e)
        return (tasks_copy.loc[taskList[0]],1e9)

    def train2(self, num_epoch, num_episodes):
        for i in num_epoch:
            probability = self.envoke_episode(num_episodes)
            returns = returns.reshape(1000, -1)
            loss = -(probs * returns).sum()
            # print(loss,self.traverse(self.agent,1))
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()  # Update the model parameters
            for name, param in self.agent.named_parameters():
                print(name, param.grad)
            print("----------------------------")
            # return

    def train(self, num_epoch,num_episodes):
        for i in range(num_epoch):
            state = tasks_copy.loc[taskList[0]]
            states = []
            rewards = []
            actions = []
            probs = []
            for j in range(num_episodes):
                x = torch.tensor(np.array(state.values, dtype=np.float32)).unsqueeze(0)
                action_logit = self.agent(x)
                action_probabilities = F.softmax(action_logit, dim=1).squeeze(0)

                action = np.random.choice(
                    [x for x in range(8)], p=action_probabilities.detach().numpy()
                )

                next_state,reward = self.execute_action(state,action)

                states.append(state)
                rewards.append(reward)
                actions.append(action)

                state = next_state
                temp = getTraverseProbabilty(self.agent, x)
                probs.append(temp * action_probabilities[action])

                loss = (temp * action_probabilities[action] * reward)

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            print(f"//// {loss} /////")
            for name, param in self.agent.named_parameters():
                pass

env = Environment()
tree = env.agent
env.train(25, 1000)

//// tensor([12.2268], grad_fn=<MulBackward0>) /////
//// tensor([14.1055], grad_fn=<MulBackward0>) /////


KeyboardInterrupt: 